# Sentiment Classification of Product Reviews Using RNNs

## Read and Explore the dataset

Explore the dataset

Data Manipulations - Data Processing

Cleaning

Tokenize the reviews

Convert this into tensor. Array

Padding for small reviews

Build a model and train the model

Predict the sentiments


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive/")

In [ ]:
%%writefile get_data.sh
if [ ! -f amazon_product_reviews_mod.csv ]; then
  wget -O amazon_product_reviews_mod.csv https://www.dropbox.com/scl/fi/imdwnlb4b617hfrepik8c/amazon_product_reviews_mod.csv?rlkey=3oafw8jsifa74v8hsvsuhcf77&dl=0
fi

Writing get_data.sh


In [ ]:
!bash get_data.sh

--2024-01-19 13:45:26--  https://www.dropbox.com/scl/fi/imdwnlb4b617hfrepik8c/amazon_product_reviews_mod.csv?rlkey=3oafw8jsifa74v8hsvsuhcf77
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uccc844767902619ac2ecf842bf8.dl.dropboxusercontent.com/cd/0/inline/CLpsXWW5eUU9Zla5j0TiIzSdeTgEQEoVoVkpgk03hHc2z5HdwslIn4mUPpEYXadYJbjd-KHRWil4aami9axQXIzOfHdFb1Z6FXRmdievDGfCMjPOikAAO-aSWyjm5PW4Y5_2E3HFZMav5aZcbbUp8cAi/file# [following]
--2024-01-19 13:45:26--  https://uccc844767902619ac2ecf842bf8.dl.dropboxusercontent.com/cd/0/inline/CLpsXWW5eUU9Zla5j0TiIzSdeTgEQEoVoVkpgk03hHc2z5HdwslIn4mUPpEYXadYJbjd-KHRWil4aami9axQXIzOfHdFb1Z6FXRmdievDGfCMjPOikAAO-aSWyjm5PW4Y5_2E3HFZMav5aZcbbUp8cAi/file
Resolving uccc844767902619ac2ecf842bf8.dl.dropboxusercontent.com (uccc844767902619ac2ecf842bf8.dl.dropboxusercontent.com)... 16

In [ ]:
#cd 'drive/My Drive'

We already downloaded the dataset from [here](https://www.kaggle.com/datasets/datafiniti/consumer-reviews-of-amazon-products?select=1429_1.csv) and saved it as a CSV file which we'll read now using Pandas.

In [ ]:
#path = 'HousePricePrediction.csv.csv'
df = pd.read_csv('amazon_product_reviews_mod.csv')

In [ ]:
df.shape

(28332, 4)

In [ ]:
df.iloc[10]

reviews.rating                                                      5
reviews.text        I find amazon basics batteries to be equal if ...
reviews.title       ... find amazon basics batteries to be equal i...
reviews.username                                             ByTXcust
Name: 10, dtype: object

In [ ]:
print(df.iloc[1000]["reviews.rating"])
print(df.iloc[1000]["reviews.text"])

1
These are terrible. Don't last Put then in various items and their life is only 1/2 half of a Duracell


In [ ]:
print(df.iloc[27162]["reviews.rating"])
print(df.iloc[27162]["reviews.text"])

5
Had to chose between Fire 10 and Fire 8. For the money, you can't go wrong with the Fire 8. Screen is good for " older eyes".


## Data Manipulations

In [ ]:
reviews_df = df[["reviews.text", "reviews.rating",]]
reviews_df.columns = ["review", "rating"]

In [ ]:
reviews_df.head(10)

,review,rating
0,I order 3 of them and one of the item is bad q...,3
1,Bulk is always the less expensive way to go fo...,4
2,Well they are not Duracell but for the price i...,5
3,Seem to work as well as name brand batteries a...,5
4,These batteries are very long lasting the pric...,5
5,Bought a lot of batteries for Christmas and th...,5
6,ive not had any problame with these batteries ...,5
7,Well if you are looking for cheap non-recharge...,5
8,These do not hold the amount of high power jui...,3
9,AmazonBasics AA AAA batteries have done well b...,4


In [ ]:
reviews_df.isnull().sum()

review    0
rating    0
dtype: int64

In [ ]:
reviews_df.dropna(inplace=True)

In [ ]:
def sentiments(rating):
    if (rating == 5) or (rating == 4):
        return "positive"
    elif rating == 3:
        return "neutral"
    elif (rating == 2) or (rating == 1):
        return "negative"

In [ ]:
reviews_df["sentiment"] = reviews_df["rating"].apply(sentiments)

In [ ]:
reviews_df.sample(10, random_state = 86, ignore_index=True)

,review,rating,sentiment
0,perfect for plane rides. the product operates ...,5,positive
1,This suited my exact needs which was a mobile ...,5,positive
2,Seem to last a long time,4,positive
3,I never owned a Kindle before. This was easy t...,5,positive
4,good batteries for the price. i will order again,5,positive
5,I realized after purchasing that the reason yo...,1,negative
6,"I do a lot of international travelling, so I h...",5,positive
7,Bought one of these two weeks before on Amazon...,5,positive
8,I reccomend this tabley yo every one. I purcha...,5,positive
9,My granddaughter loved her tablet and the colo...,5,positive


In [ ]:
sentiments = reviews_df["sentiment"].values
reviews = reviews_df["review"].values
print(sentiments[0:5])
print("\n")
print(reviews[0:5])

['neutral' 'positive' 'positive' 'positive' 'positive']


['I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.'
 'Bulk is always the less expensive way to go for products like these'
 'Well they are not Duracell but for the price i am happy.'
 'Seem to work as well as name brand batteries at a much better price'
 'These batteries are very long lasting the price is great.']


## Convert text categories into one-hot encoded vector

We are importing keras utils here, this will be needed to convert the labels to one-hot encoded vectors. We'll use the the function "to_categorical" from this library to do this. To know more about this function, please check out this [link](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical).

1. Tensorflow
2. PyTorch
3. Keras

%pip install tensorflow

In [ ]:
import tensorflow.keras.utils as ku


Keras to-categorical function takes numerical arguments, that's why we need to convert the text labels to numerical values first. To achieve this, we'll define this funnction.

In [ ]:
def encode_sentiments(sentiment):
    if sentiment == "negative":
        return 0
    elif sentiment == "neutral":
        return 1
    elif sentiment == "positive":
        return 2

In [ ]:
label_encoding = {0: 'negative', 1: 'neutral', 2: 'positve'}

In [ ]:
sentiments_encoded = [encode_sentiments(sentiment) for sentiment in sentiments]

In [ ]:
print(sentiments[0:10])
print(sentiments_encoded[0:10])

['neutral' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'neutral' 'positive']
[1, 2, 2, 2, 2, 2, 2, 2, 1, 2]


In [ ]:
labels = ku.to_categorical(sentiments_encoded, num_classes = 3)

In [ ]:
print(labels[0:20])
sentiments[0:20]

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


array(['neutral', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'neutral', 'positive',
       'positive', 'neutral', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'positive', 'neutral'],
      dtype=object)

## Cleaning of the review texts

In [ ]:
test_str = "I am really really impressed by this product."
print(test_str)
old_pattern = "really"
new_pattern = "very"
new_str = re.sub(old_pattern, new_pattern, test_str)
print(new_str)

I am really really impressed by this product.
I am very very impressed by this product.


In [ ]:
test_str = "Python 3.8"
print(test_str)
old_pattern = r'\d'
new_pattern = '<digit>'
new_str = re.sub(old_pattern, new_pattern, test_str)
print(new_str)

Python 3.8
Python <digit>.<digit>


### Remove hyperlinks

In [ ]:
test_str = "Visit https://www.amazon.com for more information on this."
test_pattern = r'http\S+'
print(test_str)
new_str = re.sub(test_pattern, " ", test_str)
print(new_str)

Visit https://www.amazon.com for more information on this.
Visit   for more information on this.


In [ ]:
def remove_hyperlinks(text):
    pattern_for_hyperlink = r'http\S+'
    return re.sub(pattern_for_hyperlink, " ", text)

In [ ]:
reviews = [remove_hyperlinks(review) for review in reviews]

In [ ]:
reviews[0:5]

['I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.',
 'Bulk is always the less expensive way to go for products like these',
 'Well they are not Duracell but for the price i am happy.',
 'Seem to work as well as name brand batteries at a much better price',
 'These batteries are very long lasting the price is great.']

### Expand contracted words

In [ ]:
def remove_contracted_words(text):
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

In [ ]:
test_str = "I can't use this product. I don't recommend this product to anyone."
print(test_str)
print(remove_contracted_words(test_str))

I can't use this product. I don't recommend this product to anyone.
I can not use this product. I do not recommend this product to anyone.


In [ ]:
reviews = [remove_contracted_words(review) for review in reviews]

In [ ]:
reviews[0:5]

['I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.',
 'Bulk is always the less expensive way to go for products like these',
 'Well they are not Duracell but for the price i am happy.',
 'Seem to work as well as name brand batteries at a much better price',
 'These batteries are very long lasting the price is great.']

### Remove everything other than letters of alphabet

In [ ]:
def remove_non_letters(text):
    antipattern = r'[^A-Za-z]+'
    return re.sub(antipattern, " ", text)

In [ ]:
reviews = [remove_non_letters(review) for review in reviews]
reviews[0:5]

['I order of them and one of the item is bad quality Is missing backup spring so I have to put a pcs of aluminum to make the battery work ',
 'Bulk is always the less expensive way to go for products like these',
 'Well they are not Duracell but for the price i am happy ',
 'Seem to work as well as name brand batteries at a much better price',
 'These batteries are very long lasting the price is great ']

In [ ]:
test_string = "I recommend      resolution with   GB of RAM     "
test_string_list = test_string.split()
test_string_list

['I', 'recommend', 'resolution', 'with', 'GB', 'of', 'RAM']

In [ ]:
' '.join(test_string_list)

'I recommend resolution with GB of RAM'

In [ ]:
test_string = "       I do not like this product     "
test_string.strip()

'I do not like this product'

In [ ]:
test_string = "I did not like Deliver Package provided by AMAZON"
test_string.lower()

'i did not like deliver package provided by amazon'

In [ ]:
def remove_spaces_and_convert_to_Lowercase(text):
    return ' '.join(text.split()).strip().lower()

In [ ]:
reviews = [remove_spaces_and_convert_to_Lowercase(review) for review in reviews]
reviews[0:5]

['i order of them and one of the item is bad quality is missing backup spring so i have to put a pcs of aluminum to make the battery work',
 'bulk is always the less expensive way to go for products like these',
 'well they are not duracell but for the price i am happy',
 'seem to work as well as name brand batteries at a much better price',
 'these batteries are very long lasting the price is great']

## Tokenization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
VOCAB_SIZE = 30000
UNK_TOK = '<UNK>'

In [ ]:
tokenizer = Tokenizer(num_words = VOCAB_SIZE, oov_token=UNK_TOK)

In [ ]:
tokenizer.fit_on_texts(reviews)

In [ ]:
sequences = tokenizer.texts_to_sequences(reviews)

In [ ]:
print(sequences[0])
print(sequences[3])
print(reviews[0])
print(reviews[3])

[3, 313, 14, 38, 4, 42, 14, 2, 252, 9, 350, 92, 9, 1348, 1410, 2172, 28, 3, 17, 5, 177, 8, 3575, 14, 3945, 5, 240, 2, 64, 55]
[138, 5, 55, 16, 53, 16, 104, 96, 15, 47, 8, 82, 88, 25]
i order of them and one of the item is bad quality is missing backup spring so i have to put a pcs of aluminum to make the battery work
seem to work as well as name brand batteries at a much better price


## Padding

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
MAX_LEN = 32 # 32 words in my review

In [ ]:
padded_sequences = np.array(pad_sequences(sequences,
                                          maxlen=MAX_LEN,
                                          padding='post',
                                          truncating='post'))

In [ ]:
print(padded_sequences[0])
print(padded_sequences[1])
print(padded_sequences[3])

[   3  313   14   38    4   42   14    2  252    9  350   92    9 1348
 1410 2172   28    3   17    5  177    8 3575   14 3945    5  240    2
   64   55    0    0]
[532   9 181   2 216 209 167   5 137   7 296  51  30   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[138   5  55  16  53  16 104  96  15  47   8  82  88  25   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


## Create the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, SimpleRNN, Flatten, Dense

In [ ]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, 16, input_length=MAX_LEN))
model.add(Bidirectional(SimpleRNN(64, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(64)))
model.add(Flatten())
model.add(Dense(24, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 16)            480000    
                                                                 
 bidirectional (Bidirection  (None, 32, 128)           10368     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               24704     
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 3)                 7

## Model training

In [ ]:
model.fit(padded_sequences, labels, epochs=10, validation_split=0.2)

Epoch 1/10
709/709 [==============================] - 39s 48ms/step - loss: 0.3465 - accuracy: 0.8986 - val_loss: 0.2190 - val_accuracy: 0.9367
Epoch 2/10
709/709 [==============================] - 30s 43ms/step - loss: 0.2288 - accuracy: 0.9258 - val_loss: 0.1760 - val_accuracy: 0.9481
Epoch 3/10
709/709 [==============================] - 31s 44ms/step - loss: 0.1710 - accuracy: 0.9447 - val_loss: 0.1572 - val_accuracy: 0.9515
Epoch 4/10
709/709 [==============================] - 33s 47ms/step - loss: 0.1184 - accuracy: 0.9621 - val_loss: 0.1245 - val_accuracy: 0.9674
Epoch 5/10
709/709 [==============================] - 32s 45ms/step - loss: 0.0848 - accuracy: 0.9731 - val_loss: 0.1019 - val_accuracy: 0.9756
Epoch 6/10
709/709 [==============================] - 31s 44ms/step - loss: 0.0669 - accuracy: 0.9794 - val_loss: 0.0913 - val_accuracy: 0.9816
Epoch 7/10
709/709 [==============================] - 30s 43ms/step - loss: 0.0494 - accuracy: 0.9848 - val_loss: 0.0914 - val_accuracy:

## Sentiment prediction using trained model

In [ ]:
positive_sample = 'ws husband and uncle they \
loved it and how easy they are to use with fantastic featuresi gave this as a christmas gift to my inla'

negative_sample = 'if ads dont bother you then this may be a decent device purchased this \
for my kid and it was loaded down with so much spam it kept loading it up making \
it slow and laggy plus the carrasoul loadout makes it hard to navigate for kids \
not very kid friendly oh you can pay to remove the ads but it wont remove them all \
buy the samsung better everything'

neutral_sample='it is terrible'

In [ ]:
sample_sequence = tokenizer.texts_to_sequences([negative_sample])[0]
sample_sequence_padded = pad_sequences([sample_sequence],
                                       maxlen=MAX_LEN,
                                       padding='post',
                                       truncating='post')

In [ ]:
label_encoding={0:'Negative', 1:'Neutral', 2:'Positive'}

In [ ]:
from sklearn.preprocessing import LabelEncoder

predictions = model.predict(sample_sequence_padded, verbose=0)
print(np.round(predictions, 3))
predicted_label = np.argmax(predictions, axis=1)[0]
print("Review:", neutral_sample)
print("Sentiment:", label_encoding[predicted_label])

[[0.659 0.332 0.009]]
Review: it is terrible
Sentiment: Negative
